In [1]:
import os
from lxml import etree
import lxml.html
from selenium import webdriver
from user_agent import generate_user_agent

import requests

In [5]:
generate_user_agent(navigator=['chrome'])

'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2850.47 Safari/537.36'

In [6]:
co = webdriver.ChromeOptions()
co.add_argument('--headless')  # 无界面模式
co.add_argument('--disable-images')
# co.add_argument('--disable-javascript') #禁用js
# co.add_argument('--proxy-server=http://ip:port') #使用代理
co.add_argument('--user-agent={}'.format(generate_user_agent(navigator=['chrome'])))

In [7]:
browser = webdriver.Chrome(chrome_options=co,service_log_path=os.path.devnull)

In [8]:
browser.get('https://www.baidu.com')

In [10]:
url = 'https://detail.tmall.com/item.htm?spm=a220m.1000858.1000725.1.447d69d2yDijtZ&id=38691444472&areaId=310100&user_id=1599634638&cat_id=2&is_b=1&rn=ce0677f5022fba5e967a0394dad81ace'

In [11]:
headers = {'cookie': 't=cf3a0df95caa6cf2551571096627dd73; _tb_token_=e1e6e4505b84e; cookie2=14464eff0c2c334c5a9eb8542f931299; cna=jKp4E/q1di4CAXJcPehBwFtr; dnk=tb5792639_2012; uc1=cookie14=UoTeOLqBgL3YuQ%3D%3D&lng=zh_CN&cookie16=V32FPkk%2FxXMk5UvIbNtImtMfJQ%3D%3D&existShop=false&cookie21=U%2BGCWk%2F7pY%2FF&tag=8&cookie15=WqG3DMC9VAQiUQ%3D%3D&pas=0; uc3=nk2=F5RARUM1CJzrI5NWIso%3D&id2=W80gAuK3N%2B65&vt3=F8dBz44u5kI1cCEsow0%3D&lg2=UtASsssmOIJ0bQ%3D%3D; tracknick=tb5792639_2012; lid=tb5792639_2012; _l_g_=Ug%3D%3D; unb=848122997; lgc=tb5792639_2012; cookie1=AVZ9mu6mI8UrtpRNEUxVkVEnYtBqvnKTCs6qzqFX9VE%3D; login=true; cookie17=W80gAuK3N%2B65; _nk_=tb5792639_2012; sg=272; csg=c9262162; pnm_cku822=098%23E1hvuvvUvbpvU9CkvvvvvjiPPFdpQjrmnLzptj3mPmPvsjlEPFzWQjiPnLswgjtEPLe5vpvhvvmv9FyCvvpvvvvvKphv8vvvvUrvpvvvvvmmZhCvmHWvvUEpphvWh9vv9DCvpvsZvvmmZhCv2noivpvUvvmv%2BooAC1AEvpvVmvvC9cXvmphvLv9BSpvjwf0DyOvO5onmsX7vACyaWDNBlwethbBYVVzhaNAwecnbTWLv0C%2BOfCuYiXVvVE6Fp%2B0x9WkXjLEc6acEKBmAVA3lYExrV4gCvpvVvvpvvhCv2QhvCvvvvvv%3D; isg=BIqKa7da5D43i2iSRDsZZnfu23bs0wxlleYqshTDO11JxyqB_Ate5dA90jMbCIZt','user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36'}

In [12]:
def gethtml(url,headers):
    r = requests.get(url,headers = headers)
    s = etree.HTML(r.text)
    return s

In [13]:
class ChromeDownloader(object):
    def __init__(self,proxy=None):
        self.proxy=proxy
    def __enter__(self):
        #打开浏览器
        self.browser = self.get_browser()
        return self.browser
    def __exit__(self,exc_type,exc_val,exc_tb):
        # 关闭浏览器
        try:
            if self.browser:
                self.browser.quit()
        except Exception as e:
            print(e)
    def get_browser(self):
        co = webdriver.ChromeOptions()
#         co.add_argument('--headless')
        co.add_argument('--disable-images')
        if self.proxy:
            co.add_argument('--proxy-server={}'.format(self.proxy)) #使用代理
        co.add_argument('--user-agent={}'.format(generate_user_agent(navigator=['chrome'])))
        browser = webdriver.Chrome(chrome_options=co,service_log_path=os.path.devnull)
        return browser
        

In [14]:
### 使用requests获取月销量
r = requests.get(url,headers = headers)
s = etree.HTML(r.text)
s.xpath('//span[contains(text(),"月销量")]/following-sibling::span/text()')

[]

In [15]:
### 使用selenuim 获取月销量
with ChromeDownloader() as browser:
    browser.get(url)
    html = browser.page_source
#     print('html got')
    s = lxml.html.fromstring(html)
#     print(s)
    print(s.xpath('//span[contains(text(),"月销量")]/following-sibling::span/text()'))

['4']


### 使用selenuim 登录微博

In [17]:
from selenium.webdriver.common.keys import Keys

In [18]:
def login_weibo(user_name,passwd):
    with ChromeDownloader() as browser:
        browser.get('https://weibo.com/')
        element = browser.find_element_by_name('loginname')
        element.send_keys(user_name)
        element = browser.find_element_by_name('password')
        element.send_keys(passwd)
        element.send_keys(Keys.RETURN)
        cookies = browser.get_cookies()
        cookie = ';'.join([e['name']+'='+e['value'] for e in cookies])
    return cookie

In [19]:
cookie = login_weibo('1553526263@qq.com','weibo.com')

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"name","selector":"loginname"}
  (Session info: chrome=66.0.3359.181)
  (Driver info: chromedriver=2.38.552518 (183d19265345f54ce39cbb94cf81ba5f15905011),platform=Mac OS X 10.13.4 x86_64)


In [20]:
co = webdriver.ChromeOptions()
# co.add_argument('--headless')  # 无界面模式
co.add_argument('--disable-images')
# co.add_argument('--disable-javascript') #禁用js
# co.add_argument('--proxy-server=http://ip:port') #使用代理
co.add_argument('--user-agent={}'.format(generate_user_agent(navigator=['chrome'])))

In [21]:
browser = webdriver.Chrome(chrome_options=co,service_log_path=os.path.devnull)

In [22]:
browser.get('https://weibo.com')

In [23]:
element = browser.find_element_by_id('loginname')

In [24]:
element.send_keys('1553526263@qq.com')

In [25]:
element = browser.find_element_by_name('password')

In [26]:
element.send_keys('weibo.com')

In [27]:
element.send_keys(Keys.RETURN)

In [33]:
element = browser.find_element_by_name('verifycode')

In [34]:
element.send_keys('nmpkb')

In [35]:
element.send_keys(Keys.RETURN)

In [36]:
cookies = browser.get_cookies()

In [37]:
cookie = ';'.join([e['name']+'='+e['value'] for e in cookies])

In [38]:
cookie

'un=1553526263@qq.com;SSOLoginState=1527169422;SCF=AqF0x6ZOafPdjcNs7EeOh_hgdNXQs-S6SOfS5XtmyG_vI4DBB4OuSsC-IHFQg06Aufjb5uC7dgNOhRwiYdnV_YQ.;ALF=1558705421;login_sid_t=4e165c41f57acee538e92a93e10a9b79;SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WFNF9YDKHiXJlKy3IvOVk8z5JpX5K2hUgL.Foz7ShM0eKM7S0-2dJLoI7yAKGHoIN._C5tt;_s_tentry=passport.weibo.com;Apache=4901510162356.051.1527169365191;YF-Page-G0=00acf392ca0910c1098d285f7eb74a11;wb_view_log=1440*9001;wvr=6;YF-Ugrow-G0=b02489d329584fca03ad6347fc915997;SUHB=0P1dNEeS6Vmnrx;SUB=_2A252ArHfDeRhGeRO71US8SnMzDmIHXVVeaQXrDV8PUNbmtBeLVTEkW9NUGbQ95VO_oOpvSHNQhjnIi1vqU-J1eIO;YF-V5-G0=c99031715427fe982b79bf287ae448f6;WBStorage=5548c0baa42e6f3d|undefined;ULV=1527169365199:1:1:1:4901510162356.051.1527169365191:;cross_origin_proto=SSL;SINAGLOBAL=4901510162356.051.1527169365191'

#### 获取cookie之后使用requests抓取网页

In [39]:
r = requests.get('https://weibo.com/u/2047317075/home?wvr=5&lf=reg',headers = {'Cookie':cookie})

In [40]:
r.status_code

200

In [41]:
'zLucky-H' in r.text

True

In [42]:
s = etree.HTML(browser.page_source)

In [43]:
s.xpath('//div[@class="WB_text W_f14"]/a//text()')

['L',
 '50元15节英语课',
 'O',
 '【30元券】微博专享限时领券',
 '#加油高考生#',
 'O',
 '网页链接',
 'O',
 '网页链接',
 'L',
 '脑力汁视频的秒拍视频',
 'O',
 '网页链接']

### 模拟登陆36kr创业项目

In [291]:
login_url = 'https://passport.36kr.com/passport/sign_in'

identity_url = 'https://rong.36kr.com/api/user/identity'

In [292]:
headers  ={
'Cookie': 'kr_stat_uuid=pjJpN25452818; Z-XSRF-TOKEN=eyJpdiI6IjJFXC9NVEJkTEhxbEpTYkZqVVg5MkJ3PT0iLCJ2YWx1ZSI6IkIwYXpRMHVDbTl0WFN5Z3BKSGhmRVRFMVF4K1NJYjNZUTZmcGQ5SStTZUdWcTdmT0NQdGNRMWt1NTdDZjJOODd4MmZ6cnRmT1ZcL3lvM0dcL1pXdlV0bmc9PSIsIm1hYyI6ImJjZWM3MDJhY2MyZmI4NGJiOTE3ZWFlY2ZlMDc1ZGJhOGE2MGM2MzFkZmZlZDlhNjk0MTI5NjRjYjg2Zjk4ZDQifQ%3D%3D; krchoasss=eyJpdiI6Ik1OWnZZNVoxZHlVT2dIdFlhXC8wS3J3PT0iLCJ2YWx1ZSI6Im1JejBqZzJ6M2g4NkVjenE2UWVHa1wvcmhjdFlPakc2aGM1M0dIODBVSDdhZFRrellqSmxTN1lDdE96RmRHYzFxQTNZU1ZQYnBXNnBkWVh3QWNxNGVPQT09IiwibWFjIjoiNThhNDU2MTA1YzI4NTc3MmM3OGNkNzE1NDUyZjQwMGFjOTFhZGUwOTYzYmUyNWEyMjQzY2Y5YTE1ZDNlMDhhMCJ9; acw_tc=AQAAAByiEgLl4AgA8RJdZVo99P4UqmXT; Hm_lvt_e8ec47088ed7458ec32cde3617b23ee3=1527169119; Hm_lpvt_e8ec47088ed7458ec32cde3617b23ee3=1527169119; download_animation=1',
'DNT': '1',
'Host': 'passport.36kr.com',
'Origin': 'https://passport.36kr.com',
'Referer': 'https://rong.36kr.com/list/detail&',
'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36'
}

In [293]:
data = {
'type': 'login',
'bind': 'false',
'needCaptcha': 'false',
'username': '15659825623',
'password': 'hou19910922',
'ok_url': 'https://rong.36kr.com/list/detail&',
'ktm_reghost': 'rong.36kr.com'
}

In [294]:
s = requests.Session()

In [295]:
r = s.post(login_url,headers=headers,data = data)

In [296]:
r.status_code

200

In [297]:
r = s.get(identity_url)

In [298]:
r.status_code

200

In [299]:
data_url = 'https://rong.36kr.com/n/api/column/0/company?sortField=HOT_SCORE&p=1'

In [300]:
r = s.get(data_url)

In [301]:
r.status_code

200

In [302]:
r.text

'{"code":0,"data":{"phase":[],"isInvestorLimit":false,"isLogin":true,"city":[],"userIsInvestor":false,"industry":[],"label":[{"cnt":125799,"name":"不限","id":0},{"cnt":12973,"name":"非TMT","id":8},{"cnt":6739,"name":"B2C","id":76},{"cnt":6339,"name":"O2O","id":1252},{"cnt":5953,"name":"广告营销","id":85},{"cnt":5905,"name":"软件开发","id":82},{"cnt":4706,"name":"生产制造","id":7946},{"cnt":4125,"name":"游戏","id":190},{"cnt":3663,"name":"大数据","id":25},{"cnt":3498,"name":"云服务","id":170},{"cnt":3465,"name":"数据服务","id":89},{"cnt":3173,"name":"垂直电商","id":283},{"cnt":2966,"name":"外包","id":160},{"cnt":2856,"name":"社区论坛","id":214},{"cnt":2745,"name":"游戏开发","id":780},{"cnt":2503,"name":"独立品牌","id":384},{"cnt":2468,"name":"管理软件","id":471},{"cnt":2460,"name":"理财","id":181},{"cnt":2405,"name":"IT服务","id":1204},{"cnt":2292,"name":"云计算","id":589},{"cnt":2253,"name":"B2B","id":74},{"cnt":2244,"name":"行业信息化","id":9890},{"cnt":2169,"name":"影视","id":187},{"cnt":2152,"name":"电商平台","id":329},{"cnt":2108,"name":"消费升级","id

### 使用selenuim登陆36kr

In [1]:
import os
from lxml import etree
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from user_agent import generate_user_agent

import time
import json

In [2]:
user_agent = generate_user_agent(navigator=['chrome'])

In [3]:
class ChromeDownloader(object):
    def __init__(self,proxy=None):
        self.proxy=proxy
    
    def __enter__(self):
        self.browser=self.get_browser()
        return self.browser
    
    def __exit__(self,exc_type,exc_val,exc_tb):
        try:
            if self.browser:
                self.browser.quit()
        except Exception as e:
            print(e)
            
    def get_browser(self):
        co = webdriver.ChromeOptions()
        co.add_argument('--headless')
        co.add_argument('--disable-images')
        # co.add_argument('--disable-javascript') #禁用js
        # co.add_argument('--proxy-server=http://ip:port') #使用代理
        co.add_argument('--user-agent={}'.format(user_agent))
        browser = webdriver.Chrome(chrome_options = co,service_log_path=os.path.devnull)
        return browser

In [4]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [271]:
def login_36kr(url,username,password):
    with ChromeDownloader() as browser:
        browser.get(url)
        wait = WebDriverWait(browser,20)
        element = wait.until(EC.presence_of_element_located((By.NAME,'username')))
        
#         element = browser.find_element_by_name('username')
        element.send_keys(username)
        element = wait.until(EC.presence_of_element_located((By.NAME,'password')))
        
#         element = browser.find_element_by_name('password')
        element.send_keys(password)
        element.send_keys(Keys.RETURN)
        time.sleep(5)
        cookies = browser.get_cookies()
        cookie = ';'.join([e['name']+'='+e['value'] for e in cookies])
#         print(cookie)
    return cookie

In [307]:
url = 'https://passport.36kr.com/pages/?ok_url=https%3A%2F%2Frong.36kr.com%2Flist%2Fdetail%26#/login'

In [273]:
cookie = login_36kr(url,'15659825623','hou19910922')

In [303]:
headers = {'Cookie':cookie,'User-Agent':generate_user_agent(navigator=['chrome'])}

data_url = 'https://rong.36kr.com/n/api/column/0/company?sortField=HOT_SCORE&p=1'

r = requests.get(data_url,headers=headers)

r.status_code

r.text

'{"code":0,"data":{"phase":[],"isInvestorLimit":false,"isLogin":true,"city":[],"userIsInvestor":false,"industry":[],"label":[{"cnt":125799,"name":"不限","id":0},{"cnt":12973,"name":"非TMT","id":8},{"cnt":6739,"name":"B2C","id":76},{"cnt":6339,"name":"O2O","id":1252},{"cnt":5953,"name":"广告营销","id":85},{"cnt":5905,"name":"软件开发","id":82},{"cnt":4706,"name":"生产制造","id":7946},{"cnt":4125,"name":"游戏","id":190},{"cnt":3663,"name":"大数据","id":25},{"cnt":3498,"name":"云服务","id":170},{"cnt":3465,"name":"数据服务","id":89},{"cnt":3173,"name":"垂直电商","id":283},{"cnt":2966,"name":"外包","id":160},{"cnt":2856,"name":"社区论坛","id":214},{"cnt":2745,"name":"游戏开发","id":780},{"cnt":2503,"name":"独立品牌","id":384},{"cnt":2468,"name":"管理软件","id":471},{"cnt":2460,"name":"理财","id":181},{"cnt":2405,"name":"IT服务","id":1204},{"cnt":2292,"name":"云计算","id":589},{"cnt":2253,"name":"B2B","id":74},{"cnt":2244,"name":"行业信息化","id":9890},{"cnt":2169,"name":"影视","id":187},{"cnt":2152,"name":"电商平台","id":329},{"cnt":2108,"name":"消费升级","id

In [318]:
d = json.loads(r.text)
d.keys()
d['data'].keys()
d['data']['pageData']

{'data': [{'brief': '新零售咖啡连锁品牌',
   'city': 101,
   'cityStr': '北京市',
   'classic': False,
   'dau': 0,
   'exposureAmount': 0,
   'followed': False,
   'funding': False,
   'hot': False,
   'id': 51163101,
   'industry': 'CONSUMER_LIFESTYLE',
   'industryStr': '消费生活',
   'industryTag': [{'expose': 0,
     'hot': 0,
     'id': 3755,
     'level': 2,
     'name': '咖啡品牌'},
    {'expose': 0, 'hot': 0, 'id': 384, 'level': 2, 'name': '独立品牌'},
    {'expose': 1,
     'hot': 1,
     'hotAt': 1477287796000,
     'id': 118,
     'level': 1,
     'name': '消费升级'},
    {'expose': 1, 'hot': 0, 'id': 6, 'level': 0, 'name': '消费生活'},
    {'expose': 0, 'hot': 0, 'id': 10892, 'level': 2, 'name': '酒水饮料'},
    {'expose': 0, 'hot': 0, 'id': 11365, 'level': 2, 'name': '食品饮料'},
    {'expose': 0, 'hot': 0, 'id': 76, 'level': 1, 'name': 'B2C'},
    {'expose': 1, 'hot': 0, 'id': 26, 'level': 0, 'name': '电商'},
    {'expose': 0, 'hot': 0, 'id': 8, 'level': 0, 'name': '非TMT'}],
   'investParts': [],
   'labelSource

### 验证码登陆

In [64]:
### 调用阿里云验证码识别接口


def get_verivy_code_by_ali(pic_path,tp='cen'):
    host = 'http://jisuyzmsb.market.alicloudapi.com'
    path = '/captcha/recognize'
    appcode = 'c5e14e8feb17494aa6cedb63f8dcb82f'
    querys = 'type={}'.format(tp)
    url = host + path + '?' + querys

    with open(pic_path,'rb') as f:
        data = f.read()
    bodys = {}
    bodys['pic'] = base64.b64encode(data)

    headers = {
        'Authorization': 'APPCODE ' + appcode,
        'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8'
    }

    r = requests.post(url,data=bodys,headers=headers)
    if r.status_code !=200:
        return False

    res = json.loads(r.text)
    if res.get('msg') != 'ok':
        return False
    return res['result']['code']


In [37]:
import os
import time
import requests
import base64
import json
from PIL import Image
from user_agent import generate_user_agent
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException

In [76]:
def recognize_verify_code(browser):
    page_path='./page.png'
    browser.save_screenshot(page_path)
    
    image = Image.open(page_path)
    verify_code_path = './verify_code.png'
    verify_image_element = browser.find_element_by_xpath('//img[@class="pass-verifyCode"]')
    # 获取图片坐标
    x1 = verify_image_element.location['x']
    y1 = vefify_image_element.location['y']
    delta_x = verify_image_element.size['width']
    delta_y = verify_image_element.size['height']
    x2 = x1+delta_x
    y2 = y1+delta_y
    box = (x1*2,y1*2,x2*2,y2*2)
    image = image.crop(box)
    image.save(vefify_code_path)
    verify_code = get_verify_code_by_ali(verify_code_path)
    
    

In [77]:
def is_logined(browser):
    if '修改资料' in browser.page_source and '修改密码' in browser.page_source:
        print('登录成功：----------------')
        return True
    return False

In [78]:
def get_cookie(browser):
    return ';'.join([e['name']+'='+e['value'] for e in browser.get_cookies()])

In [79]:
def change_verify_pic(browser):
    element = browser.find_element_by_xpath('//a[@class="pass-change-verifyCode"]')
    element.click()
    time.sleep(1)

In [80]:
def get_baidu_cookie(user_name,password):
    with ChromeDownloader() as browser:
        url = 'https://passport.baidu.com/v2/?login'
        browser.get(url)
        # 等待元素加载完毕
        wait = WebDriverWait(browser,10)
        element = wait.until(EC.presence_of_element_located((By.XPATH,'//p[@title="用户名登录"]')))
        if 'none' not in element.get_attribute('style'):
            element.click()
            time.sleep(2)
        element = wait.until(EC.presence_of_element_located((By.NAME,'userName')))
        element.send_keys(user_name)
        element = wait.until(EC.presence_of_element_located((By.NAME,'password')))
        element.send_keys(password)
        element.send_keys(Keys.RETURN)
        if is_logined(browser):
            return get_cookie(browser)
        try:
            element = browser.find_element_by_name('verfiCode')
        except NoSuchElementException:
            element = browser.find_element_by_xpath('//input[@value="登录"]')
            element.click()
            if is_logined(browser):
                return get_cookie(browser)
            else:
                raise ValueError
        for i in range(3):
            verify_code = recognize_verify_code(browser)
            if verify_code:
                break
            change_verify_pic(browser)

        else:
            raise Exception('can not get right code')

        print('获取验证码成功：',verify_code)

        element.send_keys(verify_code)
        element.send_keys(Keys.RETURN)
        time.sleep(3)

        if is_logined(browser):
            return get_cookie(browser)
        return ''

### 抓取天猫信息

In [99]:
login_url = 'https://login.tmall.com/'

In [100]:
browser = webdriver.Chrome(chrome_options=co,service_log_path=os.path.devnull)

In [101]:
browser.get(login_url)

In [113]:
browser.quit()

In [102]:
wait = WebDriverWait(browser,10)

In [110]:
from copyheaders import headers_raw_to_dict

In [215]:
headers = {
    'origin': 'https://login.taobao.com',
    'referer':'https://login.taobao.com/member/login.jhtml?tpl_redirect_url=https%3A%2F%2Fwww.tmall.com%2F&style=miniall&enup=true&newMini2=true&full_redirect=true&sub=true&from=tmall&allp=assets_css%3D3.0.10/login_pc.css&pms=1527343465936',
    'cookie':'_uab_collina=152639900798042789164507; t=cf3a0df95caa6cf2551571096627dd73; cookie2=14464eff0c2c334c5a9eb8542f931299; _tb_token_=e1e6e4505b84e; cna=jKp4E/q1di4CAXJcPehBwFtr; lid=tb5792639_2012; tg=0; enc=pTRJpB2Uw4nUb5TUU8VewTIbuE8L2a9hrMfJ%2FVTOcq9Z62LAcHzlovHKPzHhg6JYKqmWE6jq01Rzm0T4AbdmKg%3D%3D; thw=cn; UM_distinctid=1637c1aff924-0f6cd035aff10a-33677f06-13c680-1637c1aff962dd; hng=CN%7Czh-CN%7CCNY%7C156; lc=VynJq%2FWwtQl9ognIPfhynnMKfg%3D%3D; ctoken=HXyqi7wrcNANb8SIsyv3rhllor; l=ApaWOYsMQrwIlivcmKRpYB0yZkeYEtpx; v=0; uc1=cookie14=UoTeOZfY3sWNVA%3D%3D&lng=zh_CN&cookie16=Vq8l%2BKCLySLZMFWHxqs8fwqnEw%3D%3D&existShop=false&cookie21=W5iHLLyFe3xm&tag=8&cookie15=V32FPkk%2Fw0dUvg%3D%3D&pas=0; uc3=nk2=F5RARUM1CJzrI5NWIso%3D&id2=W80gAuK3N%2B65&vt3=F8dBz491yAkkx65ecFM%3D&lg2=VT5L2FSpMGV7TQ%3D%3D; existShop=MTUyNzM0NTAyNA%3D%3D; lgc=tb5792639_2012; tracknick=tb5792639_2012; dnk=tb5792639_2012; csg=92d9c859; mt=np=&ci=0_0; log=lty=Ug%3D%3D; skt=088fba433a31ec15; _cc_=V32FPkk%2Fhw%3D%3D; cookieCheck=8369; _umdata=65F7F3A2F63DF0202CA388D0D3534B570ECA7B018C25D6EFA20A7D5A2D862319EDACDA06D70C2567CD43AD3E795C914C80CCBEF51A17A6F3D13A3D3C12318D33; isg=BOLiWMGVzTco1tFoBFK4MdfaM26EmyH9DS5yuix7btUA_4F5FcPmXYo9KjsDM17l',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36'
}

In [137]:
data_url = 'https://list.tmall.com/search_shopitem.htm?spm=a220m.1000858.1000725.2.156e44baydoP6n&user_id=3676232520&from=_1_&stype=search'

In [138]:
r = requests.get(data_url,headers = headers)

In [139]:
r.status_code

200

In [140]:
s = etree.HTML(r.text)

In [142]:
'HM官方旗舰店' in r.text

True

In [216]:
def GetProductList(url):
    r = requests.get(url,headers = headers)
    s = etree.HTML(r.text)
    
    ### 获取商品列表
    price = s.xpath('//div[@class="product"]//p[@class="productPrice"]//em/text()')
    title = s.xpath('//div[@class="product"]//p[@class="productTitle"]//a/@title')
    title_url = ['https:'+i for i in s.xpath('//div[@class="product"]//p[@class="productTitle"]//a/@href')]
    month_gc = s.xpath('//div[@class="product"]//p[@class="productStatus"]//em/text()')
    return title,title_url,price,month_gc

In [217]:
title_ls,title_url_ls,price_ls,month_gc_ls = [],[],[],[]

In [218]:
i = 0
while True:
    print('Downloading page {}'.format(i))
    url = 'https://list.tmall.com/search_shopitem.htm?spm=a220m.1000858.1000725.2.156e44baydoP6n&user_id=3676232520&from=_1_&stype=search&s={}'.format(i*60)
    title,title_url,price,month_gc = GetProductList(url)
    if title not in title_ls:
        title_ls.append(title)
        title_url_ls.append(title_url)
        price_ls.append(price)
        month_gc_ls.append(month_gc)
        i+=1
        time.sleep(1)
    else:
        break

### 获取详细页

In [200]:
title_url[0]

'https://detail.tmall.com/item.htm?id=564969451919&is_b=1&cat_id=2&q=&rn=9df61857810c993fd35f534309226043'

In [202]:
r = requests.get(title_url[0],headers={'User-Agent':generate_user_agent(navigator=['chrome'])})

In [203]:
r.status_code

200

In [205]:
s = etree.HTML(r.text)

In [206]:
s.xpath('//div[@class="rate-grid"]')

[]